<a href="https://colab.research.google.com/github/midhun-james/gliner-desc/blob/main/rag_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai langchain-google-genai langchain chromadb sentence-transformers

In [5]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 44.5 MB/s eta 0:00:00


In [3]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Configure the API key
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

# Instantiate the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [6]:
import fitz
pdf_path='assign_3_data.txt'
doc=fitz.open(pdf_path)
text=''
for page in doc:
  text+=page.get_text()
print(text[:500])

 
 
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 10-K
 
 
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Fiscal Year Ended June 30, 2022
 
 
 
OR
 
 
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Transition Period From                  to
Commission File Number 001-37845
 
MICROSOFT CORPORATION
 
 
Washington
 
91-1144442
(STATE OF INCORPORATION)
 
(I


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks=text_splitter.split_text(text)



In [13]:
print(f'Total chunks: {len(chunks)}')
print(f'first chunk : {chunks[0]}')

Total chunks: 286
first chunk : UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 10-K
 
 
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Fiscal Year Ended June 30, 2022
 
 
 
OR
 
 
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Transition Period From                  to
Commission File Number 001-37845
 
MICROSOFT CORPORATION
 
 
Washington
 
91-1144442
(STATE OF INCORPORATION)
 
(I.R.S. ID)
 
ONE MICROSOFT WAY, REDMOND, Washington 98052-6399
(425) 882-8080
www.microsoft.com/investor
 
 
 
 
 
Securities registered pursuant to Section 12(b) of
the Act:
 
 
 
 
 
 
 
 
 
Title of each class
 
Trading Symbol
 
Name of exchange on which registered
 
 
 
 
 
Common stock, $0.00000625 par value per share
 
MSFT
 
Nasdaq
3.125% Notes due 2028
 
MSFT
 
Nasdaq
2.625% Notes due 2033
 
MSFT
 
Nasdaq
 
 
 
 
 
Securities registered pursuant to Section 12(g

In [16]:
docs = text_splitter.create_documents([text])
print(f"Total documents: {len(docs)}")
print(docs[0].page_content[:300])

Total documents: 286
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 10-K
 
 
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Fiscal Year Ended June 30, 2022
 
 
 
OR
 
 
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURIT
